In [1]:
## Imports

import numpy as np
from custom_envs.mountain_car import MountainCarEnv

from function_approximators.function_approximators import NeuralNetwork, LinearModel, DecisionTree, RandomForest, SupportVectorRegressor, KNeighboursRegressor, GaussianProcess, OnlineGaussianProcess
from sklearn.gaussian_process.kernels import RBF
from sklearn.metrics.pairwise import rbf_kernel

from utils.train_utils_mc import train, solve, train_time

from agents.ad_agents import DQNAgent, LinearAgent, FQIAgent, OnlineGaussianProccessAgent

import operator


In [2]:
## Environment

function_approximators = [NeuralNetwork, LinearModel, DecisionTree, RandomForest, SupportVectorRegressor, KNeighboursRegressor, GaussianProcess, OnlineGaussianProcess]

agents = [DQNAgent, LinearAgent, *[FQIAgent]*5, OnlineGaussianProccessAgent]

RENDER = False
env = MountainCarEnv()

In [6]:
# DQN Config
CONFIG_DQN = {
    "episode_length": 200,
    "max_timesteps": 50000,
    "max_time": 30 * 60,
    "eval_freq": 2500, 
    "eval_episodes": 1,
    "learning_rate": 0.005,
    "hidden_size": (16,16),
    "target_update_freq": 100,
    "batch_size": 32,
    "gamma": 0.99,
    "buffer_capacity": int(1e6),
    "plot_loss": False,
    "epsilon": 1,
    "max_deduct": 0.95,
    "decay": 0.4,
    "lr_step_size": 1000,
    "lr_gamma": 0.99,
    "max_steps": 200,
    "non_param": False,
}

# Linear Config
CONFIG_LINEAR = {
    "episode_length": 200,
    "max_timesteps": 50000,
    "max_time": 30 * 60,
    "eval_freq": 2500, 
    "eval_episodes": 10,
    "learning_rate": 0.03,
    "target_update_freq": 200,
    "batch_size": 64,
    "gamma": 0.99,
    "buffer_capacity": int(1e7),
    "plot_loss": False,
    "epsilon": 1,
    "max_steps": 200,
    "poly_degree": 2,
    "max_deduct": 0.97,
    "decay": 0.5,
    "lr_step_size": 1000,
    "lr_gamma": 0.99,
    "non_param": False,
}

# Decision Tree Config
CONFIG_DT = {
    "episode_length": 200,
    "max_timesteps": 50000,
    "max_time": 30 * 60,
    "eval_freq": 2500, 
    "eval_episodes": 1,
    "model_save_freq": 100000,
    "model_save_capacity": 1,
    "update_freq": 1,
    "batch_size": 1024,
    "gamma": 0.99,
    "buffer_capacity": int(1e5),
    "epsilon": 1,
    "max_deduct": 0.95,
    "decay": 0.5,
    "max_steps": 200,
    "non_param": True,
    "model_params": {"criterion":"mse","max_depth": 10, "min_samples_split": 20, "min_samples_leaf": 5},
    "feature_names": ["Cart Position", "Cart Velocity", "Pole Angle", "Pole Angular Velocity", "Action: Push Left", "Action: Push Right"],
    "plot_name": "dt_depth=8",
}

# Random Forest Config
CONFIG_RF = {
    "episode_length": 200,
    "max_timesteps": 50000,
    "max_time": 30 * 60,
    "eval_freq": 2500, 
    "eval_episodes": 1,
    "model_save_freq": 2500,
    "model_save_capacity": 20,
    "update_freq": 5,
    "batch_size": 512,
    "gamma": 0.99,
    "buffer_capacity": int(1e6),
    "epsilon": 1,
    "max_deduct": 0.95,
    "decay": 0.5,
    "max_steps": 200,
    "non_param": True,
    "model_params": {"n_estimators": 5,"max_depth": 5, "min_samples_split": 20, "min_samples_leaf": 5},
}

# Support Vector Regressor Config
CONFIG_SVR = {
    "episode_length": 200,
    "max_timesteps": 50000,
    "max_time": 30 * 60,
    "eval_freq": 2500, 
    "eval_episodes": 1,
    "model_save_freq": 100000,
    "model_save_capacity": 1,
    "update_freq": 1,
    "batch_size": 512,
    "gamma": 0.99,
    "buffer_capacity": int(1e4),
    "epsilon": 1,
    "max_deduct": 0.95,
    "decay": 0.4,
    "max_steps": 200,
    "non_param": True,
    "model_params": {"kernel":"rbf", "degree": 2, "C": 1},
}


# K-Neighbors Regressor Config
CONFIG_KNR = {
    "episode_length": 200,
    "max_timesteps": 50000,
    "max_time": 30 * 60,
    "eval_freq": 2500, 
    "eval_episodes": 1,
    "model_save_freq": 2500,
    "model_save_capacity": 20,
    "update_freq": 1,
    "batch_size": 256,
    "gamma": 0.99,
    "buffer_capacity": int(1e6),
    "epsilon": 1,
    "max_deduct": 0.95,
    "decay": 0.4,
    "max_steps": 200,
    "non_param": True,
    "model_params": {"n_neighbors":7, "weights": "distance", "algorithm": "auto", "leaf_size": 30},
}

# Gaussian Process Config
CONFIG_GP = {
    "episode_length": 200,
    "max_timesteps": 20000,
    "max_time": 30 * 60,
    "eval_freq": 1000, 
    "eval_episodes": 5,
    "model_save_freq": 1000,
    "model_save_capacity": 20,
    "update_freq": 10,
    "batch_size": 512,
    "gamma": 0.99,
    "buffer_capacity": int(1e6),
    "epsilon": 1,
    "max_deduct": 0.95,
    "decay": 0.3,
    "max_steps": 200,
    "non_param": True,
    "model_params": {"alpha": 1e-10, "normalize_y": False, "kernel":  RBF(length_scale=0.08, length_scale_bounds="fixed")},
}

# Online Gaussian Process Config
CONFIG_GP_Online = {
    "episode_length": 200,
    "max_timesteps": 50000,
    "max_time": 30 * 60,
    "eval_freq": 2500, 
    "eval_episodes": 1,
    "gamma": 0.99,
    "buffer_capacity": int(1e6),
    "batch_size": 32,
    "epsilon": 1,
    "max_deduct": 0.95,
    "decay": 0.3,
    "max_steps": 200,
    "non_param": True,
    "model_params": {"sigma_0": 0.5, "init": -100, "kernel":  rbf_kernel, "epsilon_tol": 0.05, "basis_limit": 1000},
}

CONFIGS = [CONFIG_DQN, CONFIG_LINEAR, CONFIG_DT, CONFIG_RF, CONFIG_SVR, CONFIG_KNR, CONFIG_GP, CONFIG_GP_Online]
onlines = [False, False, False, False, False, False, False, True]
models = ["Neural Network", "Linear Model", "Decision Tree", "Random Forest", "Support Vectors", "K-Neighbours", "Gaussian Process", "Gaussian Process Online"]

In [7]:
## Performance Evaluation

returns = []
train_returns = []
train_times = []
n_seeds=1

j=0
for i in range(n_seeds):
    print(f"\n Run: {i+1} \n")
    r, _, t, times = train(env, 
            CONFIGS[j], 
            fa=function_approximators[j], 
            agent = agents[j], 
            render=RENDER,
            online=onlines[j],
            threshold=-1)
    env.close()
    returns.append(r)
    train_returns.append(t)
    train_times.append(times)
    



 Run: 1 



  6%|▌         | 2800/50000 [00:01<00:33, 1416.97it/s]

Evaluation at timestep 2600 returned a mean returns of -200.0
Epsilon = 0.886
Learning rate = 0.0049005


 10%|█         | 5200/50000 [00:03<00:32, 1397.19it/s]

Evaluation at timestep 5000 returned a mean returns of -200.0
Epsilon = 0.772
Learning rate = 0.00480298005


...

Evaluation at timestep 45000 returned a mean returns of -200.0
Epsilon = 0.050000000000000044
Learning rate = 0.003213058010423591


 96%|█████████▌| 47800/50000 [00:34<00:01, 1289.44it/s]

Evaluation at timestep 47600 returned a mean returns of -200.0
Epsilon = 0.050000000000000044
Learning rate = 0.003117626974456


100%|██████████| 50000/50000 [00:35<00:00, 1391.69it/s]

Evaluation at timestep 50000 returned a mean returns of -200.0
Epsilon = 0.050000000000000044
Learning rate = 0.0030555861976643253


In [8]:
train_returns


[array([-200., -200., -200., -200., -200., -200., -200., -200., -200.,
        -200., -200., -200., -200., -200., -200., -200., -200., -200.,
        -200., -200., -200., -200., -200., -200., -200., -200., -200.,
        -200., -200., -200., -200., -200., -200., -200., -200., -200.,
        -200., -200., -200., -200., -200., -200., -200., -200., -200.,
        -200., -200., -200., -200., -200., -200., -200., -200., -200.,
        -200., -200., -200., -200., -200., -200., -200., -200., -200.,
        -200., -200., -200., -200., -200., -200., -200., -200., -200.,
        -200., -200., -200., -200., -200., -200., -200., -200., -200.,
        -200., -200., -200., -200., -200., -200., -200., -200., -200.,
        -200., -200., -200., -200., -200., -200., -200., -200., -200.,
        -200., -200., -200., -200., -200., -200., -200., -200., -200.,
        -200., -200., -200., -200., -200., -200., -200., -200., -200.,
        -200., -200., -200., -200., -200., -200., -200., -200., -200.,
      

In [5]:
with open(f'simplegrid_eval_{models[j]}.csv', 'ab') as eval:
    for i in range(n_seeds):
        np.savetxt(eval, [returns[i]], delimiter=',')

In [6]:
with open(f'simplegrid_train_{models[j]}.csv', 'ab') as train:
    for i in range(n_seeds):
        np.savetxt(train, [train_returns[i]], delimiter=',')
        np.savetxt(train, [train_times[i]], delimiter=',')

In [20]:
## Sample Efficiency Evaluation

n_eps = []
n_steps = []
not_solved = []
n_seeds=30

j=7
for i in range(n_seeds):
    print(f"\n Run: {i+1} \n")
    s, e, n = solve(env, 
            CONFIGS[j], 
            fa=function_approximators[j], 
            agent = agents[j],
            target_return=-6,
            op=operator.ge, 
            render=RENDER,
            online=onlines[j])
    env.close()
    n_eps.append(e)
    n_steps.append(s)
    not_solved.append(n)


 Run: 1 

Ep. timesteps: 6
Total timesteps: 762
Total episodes: 22
Evaluation mean return: -5.999999999999999

 Run: 2 

Ep. timesteps: 6
Total timesteps: 728
Total episodes: 19
Evaluation mean return: -5.999999999999999

 Run: 3 

Ep. timesteps: 6
Total timesteps: 282
Total episodes: 7
Evaluation mean return: -5.999999999999999

 Run: 4 

Ep. timesteps: 6
Total timesteps: 425
Total episodes: 10
Evaluation mean return: -5.999999999999999

 Run: 5 

Ep. timesteps: 6
Total timesteps: 282
Total episodes: 10
Evaluation mean return: -5.999999999999999

 Run: 6 

Ep. timesteps: 6
Total timesteps: 710
Total episodes: 18
Evaluation mean return: -5.999999999999999

 Run: 7 

Ep. timesteps: 6
Total timesteps: 1075
Total episodes: 25
Evaluation mean return: -5.999999999999999

 Run: 8 

Ep. timesteps: 6
Total timesteps: 521
Total episodes: 19
Evaluation mean return: -5.999999999999999

 Run: 9 

Ep. timesteps: 6
Total timesteps: 412
Total episodes: 12
Evaluation mean return: -5.999999999999999



In [21]:
with open(f'simplegrid_se-6_{models[j]}.csv', 'ab') as se:
    np.savetxt(se, [n_eps], delimiter=',')
    np.savetxt(se, [n_steps], delimiter=',')
    np.savetxt(se, [not_solved], delimiter=',')

In [22]:
mean_eps = np.mean(n_eps)
std_eps = np.std(n_eps)
print(f"Average n_eps: {mean_eps}")
print(f"Std n_eps: {std_eps}")
print(f"St.error n_eps: {std_eps/np.sqrt(n_seeds)}")

mean_steps = np.mean(n_steps)
std_steps = np.std(n_steps)
print(f"Average n_steps: {mean_steps}0")
print(f"Std n_steps: {std_steps}")
print(f"St.error n_steps: {std_steps/np.sqrt(n_seeds)}")

print(f"Not solved: {np.sum(not_solved)} runs")

Average n_eps: 21.633333333333333
Std n_eps: 27.45721924902245
St.error n_eps: 5.0129794496848845
Average n_steps: 900.60
Std n_steps: 1383.9435826651315
St.error n_steps: 252.67237284673604
Not solved: 3 runs


In [23]:
## Training time

times = []
for j in range(8):
        time = train_time(env, 
                CONFIGS[j], 
                fa=function_approximators[j], 
                agent = agents[j],
                online=onlines[j])
        env.close()
        times.append(time)

print(time)

5005it [00:06, 732.22it/s]                          
  3%|▎         | 173/5000 [00:00<00:02, 1637.78it/s]-6
5006it [00:03, 1297.35it/s]                          
 11%|█         | 549/5000 [00:00<00:01, 3684.31it/s]-6
5007it [02:02, 40.97it/s]
  0%|          | 0/5000 [00:00<?, ?it/s]-8
5001it [03:33, 23.42it/s]
  0%|          | 0/5000 [00:00<?, ?it/s]-6
5003it [09:31,  8.75it/s]
  1%|          | 28/5000 [00:00<00:20, 246.46it/s]-6
5004it [00:56, 89.35it/s]                          -6
56.0056312084198



In [24]:
with open(f'simplegrid_times.csv', 'ab') as t:
    np.savetxt(t, [times], delimiter=',')